# 02 - Tuning Parametri HSV

Notebook interattivo per ottimizzare i parametri di detection HSV.

In [ ]:
import sys
sys.path.insert(0, '..')

import cv2
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
from pathlib import Path

%matplotlib inline

## 1. Carica Frame

In [ ]:
VIDEO_PATH = '../data/videos/input/video1.mp4'

cap = cv2.VideoCapture(VIDEO_PATH)
ret, frame = cap.read()
cap.release()

if ret:
    print(f"Frame caricato: {frame.shape}")
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
else:
    print("Errore caricamento")

## 2. Visualizza Canali HSV

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Originale (RGB)')
axes[0, 0].axis('off')

axes[0, 1].imshow(hsv_frame[:,:,0], cmap='hsv')
axes[0, 1].set_title('Hue (H)')
axes[0, 1].axis('off')

axes[1, 0].imshow(hsv_frame[:,:,1], cmap='gray')
axes[1, 0].set_title('Saturation (S)')
axes[1, 0].axis('off')

axes[1, 1].imshow(hsv_frame[:,:,2], cmap='gray')
axes[1, 1].set_title('Value (V)')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("\nStatistiche HSV:")
print(f"  H range: [{hsv_frame[:,:,0].min()}, {hsv_frame[:,:,0].max()}]")
print(f"  S range: [{hsv_frame[:,:,1].min()}, {hsv_frame[:,:,1].max()}]")
print(f"  V range: [{hsv_frame[:,:,2].min()}, {hsv_frame[:,:,2].max()}]")

## 3. Tuning Interattivo Range Rosso

In [2]:
# =========================
# IMPORT E SETUP
# =========================

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

print("✓ Import completati")
print(f"  OpenCV: {cv2.__version__}")
print(f"  NumPy: {np.__version__}")

# Verifica directory
VIDEO_DIR = '/app/data/videos/input/'
if os.path.exists(VIDEO_DIR):
    videos = [f for f in os.listdir(VIDEO_DIR) if f.endswith(('.mp4','.avi','.mov'))]
    print(f"✓ Directory video trovata: {len(videos)} file")
else:
    print(f"❌ Directory non trovata: {VIDEO_DIR}")





# =========================
# DETECTION FUNCTIONS
# =========================

def detect_headlights_and_plate(frame, config, debug_overlay=None):
    """
    Esegue la detection completa di fari e targa.
    Restituisce: (success, fari, extremes, plate_corners_global, backup_points, debug_roi)
    """
    height, width = frame.shape[:2]
    
    # DETECTION FARI
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    V = hsv[:,:,2]
    
    mask = cv2.inRange(V, config['V_LOWER'], 255)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,9))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    
    contours,_ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    MIN_AREA = config['MIN_CONTOUR_AREA_RATIO'] * width * height
    MAX_Y = int(height * config['MAX_Y_RATIO'])
    
    candidates = []
    for c in contours:
        if cv2.contourArea(c) < MIN_AREA:
            continue
        x,y,w,h = cv2.boundingRect(c)
        if y > MAX_Y or w/h > 1.0 or h/w < config['MIN_VERTICAL_RATIO']:
            continue
        M = cv2.moments(c)
        if M['m00'] == 0:
            continue
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        candidates.append((c,cx,cy))
    
    if len(candidates) < 2:
        return False, None, None, None, None, None
    
    candidates = sorted(candidates, key=lambda p:p[1])
    mid = len(candidates)//2
    clusters = [candidates[:mid], candidates[mid:]]
    
    # Centro fari
    fari = []
    for cluster in clusters:
        xs,ys,a = [],[],[]
        for c,cx,cy in cluster:
            ar = cv2.contourArea(c)
            xs.append(cx*ar)
            ys.append(cy*ar)
            a.append(ar)
        fari.append((int(sum(xs)/sum(a)), int(sum(ys)/sum(a))))
    
    # Punti estremi fari
    extremes = {}
    for idx, cluster in enumerate(clusters):
        y_mid = int(np.mean([cy for _,_,cy in cluster]))
        pts = []
        for c,_,_ in cluster:
            for px,py in c[:,0,:]:
                if abs(py - y_mid) <= config['Y_TOLERANCE']:
                    pts.append((px,py))
        if idx == 0:
            extremes['SX'] = min(pts, key=lambda p:p[0])
        else:
            extremes['DX'] = max(pts, key=lambda p:p[0])
    
    # DETECTION TARGA
    fari_bottom_y = []
    for cluster in clusters:
        ys = []
        for c,_,_ in cluster:
            ys.extend(c[:,0,1])
        fari_bottom_y.append(max(ys))
    
    y_base = max(fari_bottom_y)
    x1 = extremes['SX'][0]
    x2 = extremes['DX'][0]
    y1 = min(y_base + 20, height-1)
    y2 = min(y1 + int(0.18 * height), height)
    
    roi = frame[y1:y2, x1:x2]
    
    if roi.size == 0:
        return False, None, None, None, None, None
    
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    V_roi = hsv_roi[:,:,2]
    
    mask_plate = cv2.inRange(V_roi, config['V_PLATE_LOW'], config['V_PLATE_HIGH'])
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,3))
    mask_plate = cv2.morphologyEx(mask_plate, cv2.MORPH_CLOSE, kernel)
    
    contours,_ = cv2.findContours(mask_plate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return False, None, None, None, None, None
    
    largest = max(contours, key=cv2.contourArea)
    mask_cluster = np.zeros_like(mask_plate)
    cv2.drawContours(mask_cluster, [largest], -1, 255, -1)
    
    ys, xs = np.where(mask_cluster > 0)
    if len(xs) == 0 or len(ys) == 0:
        return False, None, None, None, None, None
    
    x_min, x_max = xs.min(), xs.max()
    y_min, y_max = ys.min(), ys.max()
    
    pad_x = int(0.25 * (x_max - x_min))
    pad_y = int(0.40 * (y_max - y_min))
    
    roi_x_min = max(0, x_min - pad_x)
    roi_x_max = min(roi.shape[1], x_max + pad_x)
    roi_y_min = max(0, y_min - pad_y)
    roi_y_max = min(roi.shape[0], y_max + pad_y)
    
    roi_plate = roi[roi_y_min:roi_y_max, roi_x_min:roi_x_max]
    
    if roi_plate.size == 0:
        return False, None, None, None, None, None
    
    # Pre-processing
    gray_plate = cv2.cvtColor(roi_plate, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    gray_plate = clahe.apply(gray_plate)
    gray_plate = cv2.GaussianBlur(gray_plate, (7,7), 0)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    gradient = cv2.morphologyEx(gray_plate, cv2.MORPH_GRADIENT, kernel)
    
    _, gradient_binary = cv2.threshold(gradient, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Usa minAreaRect come fallback robusto
    contours_grad, _ = cv2.findContours(gradient_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours_grad:
        return False, None, None, None, None, None
    
    main_contour = max(contours_grad, key=cv2.contourArea)
    
    # minAreaRect (più robusto)
    rect = cv2.minAreaRect(main_contour)
    box = cv2.boxPoints(rect)
    box = np.int32(box)
    
    # Ordina punti (TL, TR, BR, BL)
    pts = box.reshape(4,2)
    s = pts.sum(axis=1)
    d = np.diff(pts, axis=1).flatten()
    
    TL = tuple(pts[np.argmin(s)])
    BR = tuple(pts[np.argmax(s)])
    
    remaining_idx = [i for i in range(4) if i not in [np.argmin(s), np.argmax(s)]]
    if pts[remaining_idx[0]][0] > pts[remaining_idx[1]][0]:
        TR = tuple(pts[remaining_idx[0]])
        BL = tuple(pts[remaining_idx[1]])
    else:
        TR = tuple(pts[remaining_idx[1]])
        BL = tuple(pts[remaining_idx[0]])
    
    plate_corners_roi = {'TL':TL,'TR':TR,'BL':BL,'BR':BR}
    
    # Converti in coordinate globali
    plate_corners_global = {}
    for k,(px,py) in plate_corners_roi.items():
        plate_corners_global[k] = (px + roi_x_min + x1, py + roi_y_min + y1)
    
    # Backup points (centri lati)
    backup_points = []
    
    debug_roi = None  # Placeholder
    
    return True, fari, extremes, plate_corners_global, backup_points, debug_roi


print("✓ Detection functions caricate")









# =========================
# TASK 1: HOMOGRAPHY SOLVER
# =========================

import cv2
import numpy as np

def solve_with_homography(plate_corners, camera_matrix, vehicle_config):
    """
    Task 1: Localizzazione usando omografia da 4 punti complanari (targa).
    Formula: [r1 r2 t] = K⁻¹ H
    
    Args:
        plate_corners: dict {'TL': (x,y), 'TR': (x,y), 'BL': (x,y), 'BR': (x,y)}
        camera_matrix: matrice K (3x3)
        vehicle_config: configurazione veicolo (coordinate 3D targa)
    
    Returns:
        (rvec, tvec, method_info)
    """
    # Punti 2D (immagine)
    image_points = np.array([
        plate_corners['TL'],
        plate_corners['TR'],
        plate_corners['BR'],
        plate_corners['BL']
    ], dtype=np.float32)
    
    # Punti 3D (targa nel sistema del veicolo)
    # Assumiamo targa su piano X=0 (posteriore)
    plate_3d = vehicle_config['license_plate_rear_corners']
    object_points = np.array([
        plate_3d['top_left'][:2],      # Solo X, Y (Z = 0 per omografia)
        plate_3d['top_right'][:2],
        plate_3d['bottom_right'][:2],
        plate_3d['bottom_left'][:2]
    ], dtype=np.float32)
    
    # Calcola omografia H
    H, status = cv2.findHomography(object_points, image_points, method=0)
    
    if H is None:
        return None, None, {'error': 'Homography calculation failed'}
    
    # Decomposizione: [r1 r2 t] = K⁻¹ H
    K_inv = np.linalg.inv(camera_matrix)
    H_norm = K_inv @ H
    
    # Normalizzazione
    lambda_ = (np.linalg.norm(H_norm[:, 0]) + np.linalg.norm(H_norm[:, 1])) / 2
    H_norm = H_norm / lambda_
    
    # Estrai colonne
    r1 = H_norm[:, 0]
    r2 = H_norm[:, 1]
    t = H_norm[:, 2]
    
    # Calcola r3 (perpendicolare al piano)
    r3 = np.cross(r1, r2)
    
    # Costruisci matrice di rotazione
    R = np.column_stack([r1, r2, r3])
    
    # Forza ortogonalità (SVD)
    U, _, Vt = np.linalg.svd(R)
    R = U @ Vt
    
    # Converti a Rodrigues
    rvec, _ = cv2.Rodrigues(R)
    tvec = t.reshape(3, 1)
    
    method_info = {
        'name': 'Homography (Task 1)',
        'points_used': 4,
        'type': 'coplanar',
        'lambda': float(lambda_)
    }
    
    return rvec, tvec, method_info


print("✓ Task 1: Homography Solver caricato")










# =========================
# PNP FULL SOLVER (6+ punti)
# =========================

def solve_with_pnp_full(fari, plate_corners, camera_matrix, dist_coeffs, vehicle_config):
    """
    Metodo completo con solvePnP usando tutti i punti disponibili.
    
    Args:
        fari: [(x1,y1), (x2,y2)] centri fari
        plate_corners: dict con 4 angoli targa
        camera_matrix: K
        dist_coeffs: coefficienti distorsione
        vehicle_config: modello 3D veicolo
    
    Returns:
        (rvec, tvec, method_info)
    """
    # Punti 2D
    image_points = []
    object_points = []
    
    # Fari
    tail_lights_3d = vehicle_config['tail_lights']
    image_points.extend([fari[0], fari[1]])
    object_points.extend([
        tail_lights_3d['left'],
        tail_lights_3d['right']
    ])
    
    # Angoli targa
    plate_3d = vehicle_config['license_plate_rear_corners']
    for corner_2d_name, corner_3d_name in [
        ('TL', 'top_left'), ('TR', 'top_right'), 
        ('BL', 'bottom_left'), ('BR', 'bottom_right')
    ]:
        if corner_2d_name in plate_corners:
            image_points.append(plate_corners[corner_2d_name])
            object_points.append(plate_3d[corner_3d_name])
    
    image_points = np.array(image_points, dtype=np.float32)
    object_points = np.array(object_points, dtype=np.float32)
    
    # solvePnP
    success, rvec, tvec = cv2.solvePnP(
        object_points, 
        image_points, 
        camera_matrix, 
        dist_coeffs,
        flags=cv2.SOLVEPNP_ITERATIVE
    )
    
    if not success:
        return None, None, {'error': 'PnP failed'}
    
    # Refine
    rvec, tvec = cv2.solvePnPRefineLM(
        object_points, 
        image_points, 
        camera_matrix, 
        dist_coeffs, 
        rvec, 
        tvec
    )
    
    # Calcola errore di riproiezione
    projected, _ = cv2.projectPoints(object_points, rvec, tvec, camera_matrix, dist_coeffs)
    projected = projected.reshape(-1, 2)
    error = np.linalg.norm(image_points - projected, axis=1).mean()
    
    method_info = {
        'name': 'PnP Full (6+ points)',
        'points_used': len(image_points),
        'type': 'over-determined',
        'reprojection_error': float(error)
    }
    
    return rvec, tvec, method_info


print("✓ PnP Full Solver caricato")












# =========================
# BBOX 3D PROJECTOR
# =========================

def project_vehicle_bbox(rvec, tvec, camera_matrix, dist_coeffs, vehicle_config):
    """
    Proietta la bounding box 3D del veicolo.
    
    Returns:
        vertices_2d: array (8, 2) con vertici proiettati
    """
    # Dimensioni veicolo
    dims = vehicle_config['dimensions']
    L = dims['length']
    W = dims['width']
    H = dims['height']
    
    # 8 vertici della bbox (origine: centro asse posteriore a suolo)
    vertices_3d = np.array([
        # Base (z=0)
        [0, -W/2, 0],      # 0: posteriore-destra
        [0, W/2, 0],       # 1: posteriore-sinistra
        [L, W/2, 0],       # 2: anteriore-sinistra
        [L, -W/2, 0],      # 3: anteriore-destra
        # Top (z=H)
        [0, -W/2, H],      # 4: posteriore-destra top
        [0, W/2, H],       # 5: posteriore-sinistra top
        [L, W/2, H],       # 6: anteriore-sinistra top
        [L, -W/2, H]       # 7: anteriore-destra top
    ], dtype=np.float32)
    
    # Proietta
    vertices_2d, _ = cv2.projectPoints(vertices_3d, rvec, tvec, camera_matrix, dist_coeffs)
    return vertices_2d.reshape(-1, 2).astype(int)


def draw_bbox_3d(frame, vertices_2d, color=(0, 255, 0), thickness=2):
    """
    Disegna la bbox 3D sul frame.
    """
    frame_copy = frame.copy()
    
    # Edges base (0-1-2-3-0)
    base_edges = [(0,1), (1,2), (2,3), (3,0)]
    for i, j in base_edges:
        cv2.line(frame_copy, tuple(vertices_2d[i]), tuple(vertices_2d[j]), color, thickness)
    
    # Edges top (4-5-6-7-4)
    top_edges = [(4,5), (5,6), (6,7), (7,4)]
    for i, j in top_edges:
        cv2.line(frame_copy, tuple(vertices_2d[i]), tuple(vertices_2d[j]), color, thickness)
    
    # Vertical edges (0-4, 1-5, 2-6, 3-7)
    for i in range(4):
        cv2.line(frame_copy, tuple(vertices_2d[i]), tuple(vertices_2d[i+4]), color, thickness)
    
    # Evidenzia posteriore (rosso)
    cv2.line(frame_copy, tuple(vertices_2d[0]), tuple(vertices_2d[1]), (0, 0, 255), thickness+1)
    cv2.line(frame_copy, tuple(vertices_2d[4]), tuple(vertices_2d[5]), (0, 0, 255), thickness+1)
    
    return frame_copy


print("✓ BBox 3D Projector caricato")









# =========================
# VEHICLE CONFIGURATION
# =========================

VEHICLE_CONFIG = {
    'dimensions': {
        'length': 3.70,
        'width': 1.74,
        'height': 1.525
    },
    'tail_lights': {
        'left': np.array([-0.27, 0.70, 0.50]),
        'right': np.array([-0.27, -0.70, 0.50])
    },
    'license_plate_rear_corners': {
        'top_left': np.array([0.0, 0.26, 0.455]),
        'top_right': np.array([0.0, -0.26, 0.455]),
        'bottom_left': np.array([0.0, 0.26, 0.345]),
        'bottom_right': np.array([0.0, -0.26, 0.345])
    }
}

# Camera (MODIFICA CON I TUOI PARAMETRI)
CAMERA_MATRIX = np.array([
    [800.0, 0, 640.0],
    [0, 800.0, 360.0],
    [0, 0, 1]
], dtype=np.float64)

DIST_COEFFS = np.zeros(5, dtype=np.float64)

print("✓ Configurazione veicolo e camera caricata")









# =========================
# MAIN SYSTEM CON AUTO-TRIGGER
# =========================

def detect_vehicle_presence(frame, config, min_red_area_ratio=0.001):
    """
    Verifica se c'è un veicolo nel frame controllando la presenza di luci rosse.
    
    Args:
        frame: Frame BGR
        config: Configurazione detection
        min_red_area_ratio: Percentuale minima di pixel rossi luminosi (default 0.1%)
    
    Returns:
        (is_present, red_pixel_count, confidence)
    """
    height, width = frame.shape[:2]
    total_pixels = width * height
    
    # Estrai canale V (luminosità)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    V = hsv[:, :, 2]
    
    # Maschera per zone luminose rosse (fari posteriori)
    mask_bright = cv2.inRange(V, config['V_LOWER'], 255)
    
    # Conta pixel luminosi
    bright_pixels = np.sum(mask_bright > 0)
    bright_ratio = bright_pixels / total_pixels
    
    # Soglia: almeno 0.1% del frame è luminoso
    min_pixels = int(total_pixels * min_red_area_ratio)
    is_present = bright_pixels > min_pixels
    
    # Confidence basata su quanta area luminosa c'è
    confidence = min(bright_ratio / (min_red_area_ratio * 3), 1.0)
    
    return is_present, bright_pixels, confidence


def wait_for_vehicle(cap, config, max_frames_to_check=300, visualize=True):
    """
    Scansiona il video fino a trovare il veicolo.
    
    Args:
        cap: VideoCapture object
        config: Configurazione detection
        max_frames_to_check: Massimo numero di frame da controllare
        visualize: Se True, mostra il progresso
    
    Returns:
        (frame_idx, frame, detection_data) o (None, None, None) se non trovato
    """
    print("\n🔍 RICERCA VEICOLO NEL VIDEO...")
    print(f"   Scansione primi {max_frames_to_check} frames...")
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    
    for frame_idx in range(max_frames_to_check):
        ret, frame = cap.read()
        if not ret:
            break
        
        # Check presenza veicolo
        is_present, red_pixels, confidence = detect_vehicle_presence(frame, config)
        
        if visualize and frame_idx % 30 == 0:
            print(f"  Frame {frame_idx}: pixels rossi={red_pixels}, confidence={confidence:.2%}")
        
        # Se troviamo presenza significativa, verifica con detection completa
        if is_present and confidence > 0.3:
            print(f"\n✓ Possibile veicolo rilevato al frame {frame_idx}")
            print(f"   Confidence: {confidence:.2%}, Pixel rossi: {red_pixels}")
            
            # Tenta detection completa
            success, fari, extremes, plate_corners, backup_points, _ = detect_headlights_and_plate(frame, config)
            
            if success:
                print(f"✅ VEICOLO CONFERMATO al frame {frame_idx}!")
                print(f"   Fari: {len(fari)} rilevati")
                print(f"   Angoli targa: {len(plate_corners)}")
                
                detection_data = {
                    'fari': fari,
                    'extremes': extremes,
                    'plate_corners': plate_corners,
                    'backup_points': backup_points
                }
                
                return frame_idx, frame, detection_data
            else:
                print(f"   Detection completa fallita, continuo ricerca...")
    
    print(f"\n❌ Nessun veicolo trovato nei primi {max_frames_to_check} frames")
    return None, None, None


def process_video_with_method(video_path, method_choice, config):
    """
    Processa video con il metodo scelto.
    Versione migliorata con auto-trigger su comparsa veicolo.
    """
    # Nome metodo
    method_names = {
        1: 'homography',
        2: 'pnp_full'
    }
    method_name = method_names[method_choice]
    
    # Output path
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    output_path = os.path.join(
        os.path.dirname(video_path).replace('input', 'output'),
        f"{video_name}_{method_name}_3d.mp4"
    )
    
    print(f"\n🎬 PROCESSING CON METODO: {method_name.upper()}")
    print(f"📹 Input: {video_path}")
    print(f"💾 Output: {output_path}")
    
    # Apri video
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"\n📊 Info video:")
    print(f"   Risoluzione: {width}x{height}")
    print(f"   FPS: {fps}")
    print(f"   Frames totali: {total_frames}")
    
    # === AUTO-TRIGGER: Attendi veicolo ===
    start_frame, first_frame, detection_data = wait_for_vehicle(cap, config, max_frames_to_check=300)
    
    if start_frame is None:
        print("\n❌ ERRORE: Veicolo non rilevato nel video!")
        print("   Suggerimenti:")
        print("   - Verifica che ci sia un veicolo con fari accesi")
        print("   - Controlla i parametri V_LOWER in config")
        print("   - Prova ad aumentare max_frames_to_check")
        cap.release()
        return
    
    fari = detection_data['fari']
    extremes = detection_data['extremes']
    plate_corners = detection_data['plate_corners']
    backup_points = detection_data['backup_points']
    
    # === Setup output video ===
    # Crea directory output se non esiste
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    # === Scrivi frames precedenti (senza bbox) ===
    print(f"\n📝 Scrittura frames iniziali ({start_frame} frames)...")
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    for i in range(start_frame):
        ret, frame = cap.read()
        if not ret:
            break
        
        # Aggiungi solo info base
        overlay = frame.copy()
        cv2.putText(overlay, f"Frame: {i}/{total_frames}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 200, 200), 2)
        cv2.putText(overlay, "WAITING FOR VEHICLE...", (10, 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 200, 200), 2)
        out.write(overlay)
    
    # === Processing con tracking ===
    print(f"\n⏳ PROCESSING dal frame {start_frame} al frame {total_frames}...")
    
    # Tracking setup
    prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
    tracked_points = {
        'faro_sx': np.array([fari[0]], dtype=np.float32),
        'faro_dx': np.array([fari[1]], dtype=np.float32),
    }
    for corner, pt in plate_corners.items():
        tracked_points[f'plate_{corner}'] = np.array([pt], dtype=np.float32)
    
    lk_params = dict(winSize=(21, 21), maxLevel=3,
                     criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))
    
    frames_processed = start_frame
    frames_with_pose = 0
    redetection_interval = 30
    frames_since_redetection = 0
    
    while frames_processed < total_frames:
        ret, frame = cap.read()
        if not ret:
            break
        
        overlay = frame.copy()
        curr_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # === TRACKING OPTICAL FLOW ===
        new_tracked = {}
        for name, pts in tracked_points.items():
            new_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, pts, None, **lk_params)
            if status[0][0] == 1:
                new_tracked[name] = new_pts
        
        tracked_points = new_tracked
        
        # Ricostruisci dati
        fari = []
        plate_corners = {}
        
        if 'faro_sx' in tracked_points:
            fari.append(tuple(tracked_points['faro_sx'][0].astype(int)))
        if 'faro_dx' in tracked_points:
            fari.append(tuple(tracked_points['faro_dx'][0].astype(int)))
        
        for corner in ['TL', 'TR', 'BL', 'BR']:
            key = f'plate_{corner}'
            if key in tracked_points:
                plate_corners[corner] = tuple(tracked_points[key][0].astype(int))
        
        # === RE-DETECTION PERIODICA ===
        frames_since_redetection += 1
        if frames_since_redetection >= redetection_interval:
            success, new_fari, new_extremes, new_plate, new_backup, _ = detect_headlights_and_plate(frame, config)
            if success:
                # Reinizializza tracking
                fari = new_fari
                plate_corners = new_plate
                tracked_points = {
                    'faro_sx': np.array([fari[0]], dtype=np.float32),
                    'faro_dx': np.array([fari[1]], dtype=np.float32),
                }
                for corner, pt in plate_corners.items():
                    tracked_points[f'plate_{corner}'] = np.array([pt], dtype=np.float32)
                
                frames_since_redetection = 0
                
                cv2.putText(overlay, "RE-DETECTION OK", (10, 90),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        
        # === CALCOLA POSA ===
        rvec, tvec, method_info = None, None, None
        
        if method_choice == 1:  # Homography
            if len(plate_corners) == 4:
                rvec, tvec, method_info = solve_with_homography(
                    plate_corners, CAMERA_MATRIX, VEHICLE_CONFIG
                )
        
        elif method_choice == 2:  # PnP Full
            if len(fari) == 2 and len(plate_corners) >= 3:
                rvec, tvec, method_info = solve_with_pnp_full(
                    fari, plate_corners, CAMERA_MATRIX, DIST_COEFFS, VEHICLE_CONFIG
                )
        
        # === VISUALIZZA BBOX 3D ===
        if rvec is not None and tvec is not None:
            vertices_2d = project_vehicle_bbox(rvec, tvec, CAMERA_MATRIX, DIST_COEFFS, VEHICLE_CONFIG)
            overlay = draw_bbox_3d(overlay, vertices_2d)
            frames_with_pose += 1
            
            # Info posa
            distance = np.linalg.norm(tvec)
            cv2.putText(overlay, f"Distance: {distance:.2f}m", (10, height - 60),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
        # === VISUALIZZA PUNTI ===
        for i, pt in enumerate(fari):
            cv2.circle(overlay, pt, 8, (0, 255, 255), -1)
            cv2.putText(overlay, 'L' if i == 0 else 'R', (pt[0]+10, pt[1]),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
        
        if len(plate_corners) == 4:
            poly = np.array([
                plate_corners['TL'], plate_corners['TR'],
                plate_corners['BR'], plate_corners['BL']
            ], dtype=np.int32)
            cv2.polylines(overlay, [poly], True, (0, 255, 0), 2)
        
        # === INFO ===
        cv2.putText(overlay, f"Frame: {frames_processed}/{total_frames}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(overlay, f"Method: {method_info['name'] if method_info else 'N/A'}", (10, 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        # Indicatore re-detection
        next_redetect = redetection_interval - frames_since_redetection
        color = (0, 255, 255) if next_redetect <= 3 else (200, 200, 200)
        cv2.putText(overlay, f"Next detect: {next_redetect}", (10, height - 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        
        out.write(overlay)
        frames_processed += 1
        prev_gray = curr_gray.copy()
        
        if frames_processed % 50 == 0:
            progress = (frames_processed - start_frame) / (total_frames - start_frame) * 100
            print(f"  📍 {frames_processed}/{total_frames} ({progress:.1f}%) - Pose: {frames_with_pose}")
    
    cap.release()
    out.release()
    
    # === SUMMARY ===
    print(f"\n{'='*60}")
    print("✅ PROCESSING COMPLETATO!")
    print(f"{'='*60}")
    print(f"📊 Statistiche:")
    print(f"   Frame iniziale veicolo: {start_frame}")
    print(f"   Frames processati: {frames_processed - start_frame}")
    print(f"   Frames con posa 3D: {frames_with_pose}")
    if frames_processed > start_frame:
        success_rate = frames_with_pose / (frames_processed - start_frame) * 100
        print(f"   Tasso successo: {success_rate:.1f}%")
    print(f"\n💾 Video salvato: {output_path}")
    print(f"{'='*60}")


print("✓ Sistema main con auto-trigger caricato")










## =========================
# ESECUZIONE
# =========================

# Configurazione detection
config = {
    'V_LOWER': 210,
    'MAX_Y_RATIO': 0.80,
    'MIN_VERTICAL_RATIO': 1.2,
    'MIN_CONTOUR_AREA_RATIO': 0.00015,
    'Y_TOLERANCE': 5,
    'V_PLATE_LOW': 150,
    'V_PLATE_HIGH': 240,
}

# Trova video
VIDEO_DIR = '/app/data/videos/input/'
videos = [f for f in os.listdir(VIDEO_DIR) if f.endswith(('.mp4','.avi','.mov'))]

if not videos:
    print("❌ Nessun video trovato!")
else:
    VIDEO_PATH = os.path.join(VIDEO_DIR, videos[0])
    print(f"📹 Video selezionato: {videos[0]}")
    
    # MENU INTERATTIVO
    method_choice = show_method_menu()
    
    # PROCESSA
    process_video_with_method(VIDEO_PATH, method_choice, config)
# ```

#---

## 🎯 **Come Usare**

# 1. **Esegui tutte le celle** in ordine (1-6)
# 2. Alla **Cella 6** apparirà il menu:
# ```
# ==============================================================
# VEHICLE 3D LOCALIZATION - METODO DI CALCOLO
# ==============================================================

# Scegli il metodo di localizzazione:

# 1. HOMOGRAPHY (Task 1)
#   - Usa 4 angoli targa (complanari)
#   - Formula: [r1 r2 t] = K⁻¹ H
#   - Limitazione: scarsa prospettiva

#2. PNP FULL (Metodo Completo)
#   - Usa 6+ punti (fari + targa)
#   - solvePnP con refinement
#   - Più robusto e accurato

#==============================================================

# Inserisci scelta (1-2):

✓ Import completati
  OpenCV: 4.8.1
  NumPy: 1.24.3
✓ Directory video trovata: 6 file
✓ Detection functions caricate
✓ Task 1: Homography Solver caricato
✓ PnP Full Solver caricato
✓ BBox 3D Projector caricato
✓ Configurazione veicolo e camera caricata
✓ Sistema main con auto-trigger caricato
📹 Video selezionato: 1.mp4


NameError: name 'show_method_menu' is not defined